##### 环境搭建
- 安装pyenv做本地环境管理
- 将本地目录设置为需要的python版本
- 安装Langchain包

#### 创建虚拟环境
python -m venv myenv

#### warning: donot use python 3.12. this notebook uses 3.11

##### python环境测试

In [5]:
print("hello world!")

hello world!


##### 依赖安装
- 安装openai的api包
- 或者使用Baichuan、ChatGLM这样的开源包

```
pip install openai langchain
```

In [ ]:
%pip install -r requirements.txt

#### 激活虚拟环境
myenv\Scripts\activate

In [7]:
import sys
print(sys.executable)


o:\Documents\langchain_AI_Agent\myenv\Scripts\python.exe


##### 引入openai key

1. Install python-dotenv package:  
   `pip install python-dotenv`

2. Create a .env file in the root of your project and add your API key:  
   `API_KEY=your_api_key_here`

3. Add `.env` to your `.gitignore` file to ensure it doesn’t get committed to your repository.


##### 从环境变量中读取

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

API_KEY = os.getenv("API_KEY")
API_PROXY = os.getenv("API_PROXY")

print("API_KEY:", API_KEY)
print("API_PROXY:", API_PROXY)


API_KEY: sk-proj-YIaWR6q6zOvJXan7W9dIT3BlbkFJkHTDMiX7t0yEF42n3uEP
API_PROXY: https://ai-yyds.com/v1


#### 运行前查看下安装情况

In [1]:
! pip show openai

In [2]:
! pip show langchain

### openai 官方SDK

In [10]:
from openai import OpenAI
import os

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("API_KEY"),
)

# Define the chat messages
messages = [
    {"role": "user", "content": "介绍下你自己"}
]

# Create a chat completion using the new API
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages
)

print(response)

ChatCompletion(id='chatcmpl-9YvxF50Ip6aow1CW4yG1hgAkQtRIH', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='您好，我是一名人工智能助手，专注于提供各种领域的知识和帮助，帮助用户解决问题和获取所需的信息。我可以回答您关于各种主题的问题，帮助您更好地理解和应对不同的情况。如果您有任何问题或需要帮助，请随时告诉我，我会尽力为您提供支持。希望能够帮助到您！', role='assistant', function_call=None, tool_calls=None))], created=1718112957, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=137, prompt_tokens=15, total_tokens=152))


### 使用langchain调用 openai



In [ ]:
from langchain.llms import OpenAI

# use API_BASE if you are in China
api_base = os.getenv("API_BASE")

api_key = os.getenv("API_KEY")

llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key=api_key,
    openai_api_base=api_base
    )

#### 提示模板（PromptTemplate）

In [1]:
from langchain.prompts import PromptTemplate
 
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

print(prompt.format(product="colorful socks"))

What is a good name for a company that makes colorful socks?


##### 起名大师

In [10]:
#起名大师
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import os

# use API_BASE if you are in China
api_base = os.getenv("API_BASE")

api_key = os.getenv("API_KEY")

llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key=api_key,
    openai_api_base=api_base
    )

prompt = PromptTemplate.from_template("你是一个起名大师,请模仿示例起3个{county}名字,比如男孩经常被叫做{boy},女孩经常被叫做{girl}")
message = prompt.format(county="中国特色的",boy="狗蛋",girl="翠花")
print(message)
llm.predict(message)

你是一个起名大师,请模仿示例起3个中国特色的名字,比如男孩经常被叫做狗蛋,女孩经常被叫做翠花


o:\Documents\langchain_AI_Agent\myenv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'\n\n男孩: 龙飞、铁柱、小虎\n女孩: 玉兰、梅香、小红梅'

##### 格式化输出

In [14]:
from langchain.schema import BaseOutputParser
#自定义class，继承了BaseOutputParser
class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("hi, bye")

['hi', 'bye']

In [16]:
#起名大师，输出格式为一个数组
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import os
from langchain.schema import BaseOutputParser

#自定义类
class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        print(text)
        return text.strip().split(",")


api_base = os.getenv("OPENAI_API_BASE")
api_key = os.getenv("API_KEY")
llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key=api_key,
    openai_api_base=api_base
    )
prompt = PromptTemplate.from_template("你是一个起名大师,请模仿示例起3个具有{county}特色的名字,示例：男孩常用名{boy},女孩常用名{girl}。请返回以逗号分隔的列表形式。仅返回逗号分隔的列表，不要返回其他内容。")
message = prompt.format(county="美国男孩",boy="sam",girl="lucy")
print(message)
strs = llm.predict(message)
CommaSeparatedListOutputParser().parse(strs)

你是一个起名大师,请模仿示例起3个具有美国男孩特色的名字,示例：男孩常用名sam,女孩常用名lucy。请返回以逗号分隔的列表形式。仅返回逗号分隔的列表，不要返回其他内容。


jack, michael, jason


['jack', ' michael', ' jason']

#### Create a chain

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

openai_api_key = os.getenv("API_KEY")
if openai_api_key is None:
    raise ValueError("API_KEY is not set")

chat = ChatOpenAI(api_key=openai_api_key, model="gpt-3.5-turbo")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

chain = prompt | chat

chat.invoke("how can langsmith help with testing?")


In [29]:
chain.invoke({"input":"how can langsmith help with testing?"})

AIMessage(content='Langsmith is a powerful tool that can greatly assist in testing various aspects of software applications. Here are some ways in which Langsmith can help with testing:\n\n1. **Automated Testing**: Langsmith can be used to automate the testing process, making it faster and more efficient. You can write scripts in Langsmith to perform various tests, such as unit tests, integration tests, and end-to-end tests.\n\n2. **Performance Testing**: Langsmith can help in conducting performance tests to evaluate the speed, responsiveness, and stability of your application under different conditions. This can help identify potential bottlenecks and optimize the performance of the software.\n\n3. **Load Testing**: Langsmith can simulate a large number of users accessing your application simultaneously, helping to determine how well your application can handle peak loads. This can be crucial in ensuring that your application can perform well under heavy traffic.\n\n4. **API Testing**